### Import custom modules from current folder

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import nltk
from text_easability_metrics import TextEasabilityMetrics, StanfordNLP
from simple_text_representation.classes import Text
from simple_text_representation.models import Database

In [3]:
database = Database('educationalTexts', 'postgres', '', '0.0.0.0', 5432)
path = r'/Users/herbert/Projects/Tesis/stanford-corenlp-full-2017-06-09'
path = r'http://corenlp.run'
path = r'http://localhost/'

In [4]:
metrics = TextEasabilityMetrics(path, port=9000)
firstTestText = Text.getText(database, 1)

TextEasabilityMetrics


### Text easability analysis

In [5]:
text = firstTestText

In [6]:
# testSentence = 'Si bien los trasplantes se han convertido en una práctica habitual, aún persisten fuertes temores en la población para donar órganos, lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.'
testSentence = text[0][0]
testSentence

# corpus = nltk.corpus.conll2002
# sentenceGrammar = corpus.chunked_sents('esp.train')[2]
# sentence= corpus.sents('esp.train')[2]

# parser = nltk.ChartParser(sentenceGrammar)
# for doc in parser.parse(sentence):
#     print(doc)


'Si bien los trasplantes se han convertido en una práctica habitual aún persisten fuertes temores en la población para donar órganos lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.'

In [7]:
parseSentence = metrics.syntaxisSimplicity(testSentence)
parseSentence

(ROOT
  (S
    (NP
      (NP
        (NP (NNP Si))
        (NP (FW bien) (FW los) (FW trasplantes) (FW se) (FW han) (FW convertido) (FW en) (FW una) (FW práctica)))
      (NP (JJ habitual) (NN aún) (NN persisten)))
    (VP (VBZ fuertes)
      (NP
        (NP (NNS temores))
        (PP (IN en)
          (NP
            (NP (DT la) (FW población) (FW para) (FW donar) (FW órganos) (FW lograr) (FW su) (FW superación) (FW es) (FW la) (FW clave) (FW para) (FW aumentar) (FW el) (FW número) (FW de) (FW los) (FW donadores) (FW solidarios) (FW que))
            (PP
              (NP (NN hacen) (NN falta) (NN para) (NN salvar) (NNS miles))
              (IN de)
              (NP (NNS vidas)))))))
    (. .)))
Syntactic Simplicity


0

In [ ]:
text = firstTestText
for paragraph in text:
    for sentence in paragraph:
        metrics.syntaxisSimplicity(sentence)